In [5]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm

# Load dataset
data = pd.read_csv('../datasets/CropSDEData/METEO_DEKADS_NUTS2_NL.csv')

# Feature Selection
target = 'PREC'

# Drop rows with missing values
data = data.dropna(subset=[target])

# Prepare data
y = data[target].values

# Method of Moments (MoM) for Vasicek Model
def vasicek_mom(y):
    dt = 1  # Assuming daily intervals

    # Calculate differences
    dy = y[1:] - y[:-1]

    # Estimate parameters
    sigma_mom = np.sqrt(np.var(dy) / dt)
    b_mom = np.mean(y)
    a_mom = -np.mean(dy) / (b_mom - np.mean(y[:-1]))

    return a_mom, b_mom, sigma_mom

# Maximum Likelihood Estimation (MLE) for Vasicek Model
def vasicek_mle(params, data):
    a, b, sigma = params
    dt = 1  # Assuming daily intervals
    X = data

    # Vasicek model residuals
    residuals = X[1:] - (X[:-1] + a * (b - X[:-1]) * dt)
    
    # Log likelihood
    log_likelihood = -np.sum(0.5 * np.log(2 * np.pi * sigma**2 * dt) + (residuals**2 / (2 * sigma**2 * dt)))
    return -log_likelihood  # Negative for minimization

# Least Squares (LSQ) for Vasicek Model
def vasicek_lsq(params, data):
    a, b, sigma = params
    dt = 1  # Assuming daily intervals
    X = data

    # Predicted values using the Vasicek model
    predicted = X[:-1] + a * (b - X[:-1]) * dt

    # Residual sum of squares
    residuals = X[1:] - predicted
    return np.sum(residuals**2)

# Bayesian Estimation for Vasicek Model (Simplified Grid Search)
def vasicek_bayesian(y):
    dt = 1  # Assuming daily intervals
    a_vals = np.linspace(0.01, 1, 50)
    b_vals = np.linspace(np.min(y), np.max(y), 50)
    sigma_vals = np.linspace(0.01, 2, 50)

    best_a, best_b, best_sigma = None, None, None
    max_posterior = -np.inf

    for a in a_vals:
        for b in b_vals:
            for sigma in sigma_vals:
                residuals = y[1:] - (y[:-1] + a * (b - y[:-1]) * dt)
                likelihood = np.sum(norm.logpdf(residuals, loc=0, scale=sigma))
                prior = norm.logpdf(a, loc=0.5, scale=0.5) + norm.logpdf(b, loc=np.mean(y), scale=np.std(y)) + norm.logpdf(sigma, loc=1, scale=0.5)
                posterior = likelihood + prior

                if posterior > max_posterior:
                    max_posterior = posterior
                    best_a, best_b, best_sigma = a, b, sigma

    return best_a, best_b, best_sigma

# Get MoM estimates
a_mom, b_mom, sigma_mom = vasicek_mom(y)

print(f"Estimated Vasicek Parameters using Method of Moments:")
print(f"Alpha (a): {a_mom}")
print(f"Beta (b): {b_mom}")
print(f"Sigma: {sigma_mom}")

# Initial guess for MLE and LSQ parameters
initial_guess = [0.1, np.mean(y), 0.1]

# Estimate Vasicek parameters using MLE
res_mle = minimize(vasicek_mle, initial_guess, args=(y,), method='L-BFGS-B', 
                   bounds=[(1e-5, None), (None, None), (1e-5, None)])

# Extract MLE parameters
a_mle, b_mle, sigma_mle = res_mle.x

print(f"\nEstimated Vasicek Parameters using Maximum Likelihood Estimation (MLE):")
print(f"Alpha (a): {a_mle}")
print(f"Beta (b): {b_mle}")
print(f"Sigma: {sigma_mle}")
print(f"Convergence Status: {res_mle.success}")
print(f"Message: {res_mle.message}")

# Estimate Vasicek parameters using LSQ
res_lsq = minimize(vasicek_lsq, initial_guess, args=(y,), method='L-BFGS-B', 
                   bounds=[(1e-5, None), (None, None), (1e-5, None)])

# Extract LSQ parameters
a_lsq, b_lsq, sigma_lsq = res_lsq.x

print(f"\nEstimated Vasicek Parameters using Least Squares (LSQ):")
print(f"Alpha (a): {a_lsq}")
print(f"Beta (b): {b_lsq}")
print(f"Sigma: {sigma_lsq}")
print(f"Convergence Status: {res_lsq.success}")
print(f"Message: {res_lsq.message}")

# Estimate Vasicek parameters using Bayesian Estimation
best_a, best_b, best_sigma = vasicek_bayesian(y)

print(f"\nEstimated Vasicek Parameters using Bayesian Estimation (Simplified):")
print(f"Alpha (a): {best_a}")
print(f"Beta (b): {best_b}")
print(f"Sigma: {best_sigma}")


Estimated Vasicek Parameters using Method of Moments:
Alpha (a): -0.424106794233483
Beta (b): 1.9206313657407403
Sigma: 2.230060821066932

Estimated Vasicek Parameters using Maximum Likelihood Estimation (MLE):
Alpha (a): 0.9685389235036493
Beta (b): 1.920622930269423
Sigma: 1.6015054801592965
Convergence Status: True
Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH

Estimated Vasicek Parameters using Least Squares (LSQ):
Alpha (a): 0.9685380244379254
Beta (b): 1.9206259587253125
Sigma: 0.1
Convergence Status: True
Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH

Estimated Vasicek Parameters using Bayesian Estimation (Simplified):
Alpha (a): 0.9595918367346938
Beta (b): 1.9542857142857144
Sigma: 1.5938775510204082
